# Processamento/tratamento dos dados

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'C:\Users\João Pedro\Documents\UFG\AMS\AS2\data\train.csv')
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
df_copy = df.copy()

## Tratando os nulos da coluna *Name* e criação de novas features

In [4]:
def processar_nomes(df):
    df_proc = df.copy()
    
    # 1. Extrair Grupo e Sobrenome inicial
    df_proc['Group'] = df_proc['PassengerId'].str.split('_', expand=True)[0]
    df_proc['Surname'] = df_proc['Name'].str.split().str[-1]
    
    # 2. Tentar preencher Sobrenome faltante usando o Grupo
    # Cria um dicionário: {Grupo: Sobrenome_Mais_Comum}
    group_surname_map = df_proc.groupby('Group')['Surname'].apply(lambda x: x.mode()[0] if not x.mode().empty else None)
    
    # Preenche onde Surname é nulo usando o mapa
    df_proc['Surname'] = df_proc['Surname'].fillna(df_proc['Group'].map(group_surname_map))
    
    # 3. Quem ainda está nulo vira "Unknown"
    df_proc['Surname'] = df_proc['Surname'].fillna('Unknown')
    
    # 4. Criar Features de Tamanho
    # Tamanho da Família (baseado em Sobrenome)
    df_proc['FamilySize'] = df_proc['Surname'].map(df_proc['Surname'].value_counts())
    
    # Tamanho do Grupo (baseado no ID - esse não tem nulos!)
    df_proc['GroupSize'] = df_proc['Group'].map(df_proc['Group'].value_counts())
    
    # 5. Limpeza Final: Remover Name (já extraímos o útil)
    # Nota: Mantemos Surname por enquanto pois ele ajuda a preencher HomePlanet/CryoSleep
    df_proc = df_proc.drop(columns=['Name'])
    
    return df_proc

# Aplicar
df_copy = processar_nomes(df_copy)

In [12]:
df_copy.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Group,Surname,FamilySize,GroupSize
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,0001,Ofracculy,1,1
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,0002,Vines,4,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,0003,Susent,6,2
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,0003,Susent,6,2
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,0004,Santantines,6,1


In [15]:
df_copy.isna().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
Group           0
Surname         0
FamilySize      0
GroupSize       0
dtype: int64

## Tratamento do resto das colunas nulas

In [14]:
def tratar_nulos(df):
    df_clean = df.copy()
    
    if 'Group' not in df_clean.columns:
        df_clean['Group'] = df_clean['PassengerId'].str.split('_', expand=True)[0]
    if 'Surname' not in df_clean.columns and 'Name' in df_clean.columns:
        df_clean['Surname'] = df_clean['Name'].str.split().str[-1]

    # --- 1. RELAÇÃO CRYOSLEEP x GASTOS ---
    gastos_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    
    # Regra A: Se CryoSleep=True, Gastos devem ser 0
    df_clean.loc[df_clean['CryoSleep'] == True, gastos_cols] = 0.0
    
    # Regra B: Se Gastos > 0, CryoSleep deve ser False
    total_gastos = df_clean[gastos_cols].sum(axis=1)
    df_clean.loc[(df_clean['CryoSleep'].isna()) & (total_gastos > 0), 'CryoSleep'] = False

    # --- 2. IMPUTAÇÃO INTELIGENTE (Grupo e Família) ---
    # Colunas alvo para preencher usando Grupo/Sobrenome
    target_cols = ['HomePlanet', 'Destination', 'VIP', 'CryoSleep'] 
    
    for col in target_cols:
        # Estratégia 1: Tentar preencher com a moda do GRUPO (PassengerId)
        # Ex: Se alguém do meu grupo é de 'Europa', eu provavelmente também sou.
        df_clean[col] = df_clean[col].fillna(
            df_clean.groupby('Group')[col].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
        )
        
        # Estratégia 2: Tentar preencher com a moda do SOBRENOME (Família)
        # Ex: Se meu grupo não ajudou, vejo minha família.
        if 'Surname' in df_clean.columns:
            df_clean[col] = df_clean[col].fillna(
                df_clean.groupby('Surname')[col].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
            )

    # --- 3. PREENCHIMENTO FINAL (Valores Globais) ---
    
    # Numéricas -> Mediana (incluindo gastos que sobraram como NA e Idade)
    num_cols = ['Age'] + gastos_cols
    for col in num_cols:
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())
    
    # Categóricas Restantes -> Moda Global
    cat_cols = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']
    for col in cat_cols:
        df_clean[col] = df_clean[col].fillna(df_clean[col].mode()[0])

    # Cabin -> Valor Genérico (Placeholder)
    df_clean['Cabin'] = df_clean['Cabin'].fillna('Z/9999/Z')
    
    return df_clean

# Aplicação
df_copy = tratar_nulos(df_copy)
# df_test_clean = tratar_nulos(df_test)

print("Nulos restantes no treino após tratamento avançado:")
print(df_copy.isnull().sum().sum())

Nulos restantes no treino após tratamento avançado:
0


In [20]:
df_copy.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Group,Surname,FamilySize,GroupSize
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,0001,Ofracculy,1,1
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,0002,Vines,4,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,0003,Susent,6,2
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,0003,Susent,6,2
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,0004,Santantines,6,1


In [24]:
def clean_bool(x):
    # Converte tudo para string primeiro, depois padroniza
    x = str(x).lower()
    if x in ['true', '1.0', '1']:
        return True
    elif x in ['false', '0.0', '0']:
        return False
    return False # Valor padrão caso sobre algo estranho

# Aplicar correção
df_copy['CryoSleep'] = df_copy['CryoSleep'].apply(clean_bool)
df_copy['VIP'] = df_copy['VIP'].apply(clean_bool)

# Verificar novamente
print(f"CryoSleep únicos: {df_copy['CryoSleep'].unique()}")
print(f"VIP únicos: {df_copy['VIP'].unique()}")

CryoSleep únicos: [False  True]
VIP únicos: [False  True]


In [25]:
df_copy.to_csv(r'C:\Users\João Pedro\Documents\UFG\AMS\AS2\data\train_cleaned.csv', index=False)

# Engenharia de features

In [2]:
df_cleaned = pd.read_csv(r'C:\Users\João Pedro\Documents\UFG\AMS\AS2\data\train_cleaned.csv')
df_cleaned.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Group,Surname,FamilySize,GroupSize
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,1,Ofracculy,1,1
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,2,Vines,4,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,3,Susent,6,2
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,3,Susent,6,2
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,4,Santantines,6,1


In [3]:
def feature_engineering(df):
    df_feat = df.copy()
    
    # 1. CABINE (Deck, Num, Side)
    # Divide a string 'B/0/P' em 3 colunas
    # Usamos n=2 para garantir que divida apenas nas duas primeiras barras (embora o formato seja fixo)
    df_feat[['Deck', 'Num', 'Side']] = df_feat['Cabin'].str.split('/', expand=True)
    
    # Converter Num para int para podermos ordenar ou criar faixas depois se quiser
    df_feat['Num'] = df_feat['Num'].astype(int)
    
    # 2. IDADE (Categorias)
    # Cria faixas etárias
    # 0-12: Criança, 13-17: Teen, 18-60: Adulto, 60+: Idoso
    df_feat['AgeGroup'] = pd.cut(df_feat['Age'], 
                                 bins=[-1, 12, 17, 60, 200], 
                                 labels=['Child', 'Teen', 'Adult', 'Senior'])
    
    # 3. GASTOS (Total e Zero)
    cols_gastos = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    df_feat['TotalSpend'] = df_feat[cols_gastos].sum(axis=1)
    df_feat['NoSpending'] = (df_feat['TotalSpend'] == 0).astype(int) # 1 se não gastou nada
    
    # 4. GRUPO (IsAlone)
    # Se GroupSize for 1, a pessoa está viajando sozinha
    df_feat['IsAlone'] = (df_feat['GroupSize'] == 1).astype(int)
    
    # 5. REMOÇÃO DE COLUNAS DESNECESSÁRIAS
    # Removemos Cabin (já extraímos dados), PassengerId e Name (já extraímos grupos/nomes)
    # Mas NÃO remova 'PassengerId' se for o arquivo de TESTE final para submissão (guarde ele separado).
    cols_drop = ['Cabin', 'Name', 'PassengerId', 'Group', 'Surname'] 
    
    # Verificação segura antes de dropar
    cols_drop = [c for c in cols_drop if c in df_feat.columns]
    df_feat = df_feat.drop(columns=cols_drop)
    
    return df_feat

# Aplicar a engenharia
df_cleaned = feature_engineering(df_cleaned)

# Visualizar as novas colunas
print(df_cleaned[['Deck', 'Side', 'Num', 'AgeGroup', 'TotalSpend', 'IsAlone']].head())

  Deck Side  Num AgeGroup  TotalSpend  IsAlone
0    B    P    0    Adult         0.0        1
1    F    S    0    Adult       736.0        1
2    A    S    0    Adult     10383.0        0
3    A    S    0    Adult      5176.0        0
4    F    S    1     Teen      1091.0        1


In [4]:
df_cleaned.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,FamilySize,GroupSize,Deck,Num,Side,AgeGroup,TotalSpend,NoSpending,IsAlone
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,1,1,B,0,P,Adult,0.0,1,1
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,4,1,F,0,S,Adult,736.0,0,1
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,6,2,A,0,S,Adult,10383.0,0,0
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,6,2,A,0,S,Adult,5176.0,0,0
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,6,1,F,1,S,Teen,1091.0,0,1


In [5]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   HomePlanet    8693 non-null   object  
 1   CryoSleep     8693 non-null   bool    
 2   Destination   8693 non-null   object  
 3   Age           8693 non-null   float64 
 4   VIP           8693 non-null   bool    
 5   RoomService   8693 non-null   float64 
 6   FoodCourt     8693 non-null   float64 
 7   ShoppingMall  8693 non-null   float64 
 8   Spa           8693 non-null   float64 
 9   VRDeck        8693 non-null   float64 
 10  Transported   8693 non-null   bool    
 11  FamilySize    8693 non-null   int64   
 12  GroupSize     8693 non-null   int64   
 13  Deck          8693 non-null   object  
 14  Num           8693 non-null   int64   
 15  Side          8693 non-null   object  
 16  AgeGroup      8693 non-null   category
 17  TotalSpend    8693 non-null   float64 
 18  NoSpendi

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

def preparar_dados_e_salvar(df_train_cleaned):
    # --- 1. Engenharia Básica (Log, etc) ---
    df_proc = df_train_cleaned.copy()
    
    # Recriar features necessárias
    if 'TotalSpend' not in df_proc.columns:
        cols_gastos = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
        df_proc['TotalSpend'] = df_proc[cols_gastos].sum(axis=1)
        
    if 'Deck' not in df_proc.columns and 'Cabin' in df_proc.columns:
        df_proc[['Deck', 'Num', 'Side']] = df_proc['Cabin'].str.split('/', expand=True)
        df_proc['Num'] = pd.to_numeric(df_proc['Num'], errors='coerce').fillna(0)

    # Log Transform
    cols_to_log = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'TotalSpend', 'Num']
    for col in cols_to_log:
        if col in df_proc.columns:
            df_proc[col] = np.log1p(df_proc[col])

    # --- 2. Split ---
    # Identificar colunas alvo e features
    cols_drop = ['PassengerId', 'Name', 'Cabin', 'Group', 'Surname', 'Transported']
    features = [c for c in df_proc.columns if c not in cols_drop]
    
    X = df_proc[features]
    y = df_proc['Transported'].astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # --- 3. Preprocessor ---
    num_cols = [c for c in X_train.columns if X_train[c].dtype in ['int64', 'float64']]
    cat_cols = [c for c in X_train.columns if X_train[c].dtype == 'object']

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_cols),
            ('cat', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False), cat_cols)
        ],
        verbose_feature_names_out=False
    )

    # Fit e Transform
    X_train_final = preprocessor.fit_transform(X_train)
    X_val_final = preprocessor.transform(X_val)
    
    # --- RECUPERAR NOMES DAS COLUNAS ---
    feature_names = preprocessor.get_feature_names_out()
    
    return X_train_final, y_train, X_val_final, y_val, feature_names

# Executar a função
# df_clean = pd.read_csv('train_cleaned.csv') # Carregar seu dataset limpo
X_train, y_train, X_val, y_val, col_names = preparar_dados_e_salvar(df_cleaned)

In [14]:
import pandas as pd
import numpy as np

# --- SALVANDO EM 4 ARQUIVOS DISTINTOS ---

# 1. Salvar X_train (Features de Treino)
# Convertendo o array numpy de volta para DataFrame para ter cabeçalho
df_X_train = pd.DataFrame(X_train, columns=col_names)
df_X_train.to_csv('X_train.csv', index=False)

# 2. Salvar y_train (Alvo de Treino)
# O y_train original é uma Series, convertemos para DataFrame
df_y_train = pd.DataFrame(y_train).reset_index(drop=True)
df_y_train.columns = ['Transported'] # Garante o nome da coluna
df_y_train.to_csv('y_train.csv', index=False)

# 3. Salvar X_val (Features de Validação)
df_X_val = pd.DataFrame(X_val, columns=col_names)
df_X_val.to_csv('X_val.csv', index=False)

# 4. Salvar y_val (Alvo de Validação)
df_y_val = pd.DataFrame(y_val).reset_index(drop=True)
df_y_val.columns = ['Transported']
df_y_val.to_csv('y_val.csv', index=False)

print("Arquivos gerados com sucesso:")
print(f"- X_train_final.csv: {df_X_train.shape}")
print(f"- y_train_final.csv: {df_y_train.shape}")
print(f"- X_val.csv:         {df_X_val.shape}")
print(f"- y_val.csv:         {df_y_val.shape}")

Arquivos gerados com sucesso:
- X_train_final.csv: (6954, 26)
- y_train_final.csv: (6954, 1)
- X_val.csv:         (1739, 26)
- y_val.csv:         (1739, 1)
